In [1]:
import OpenDartReader
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from html_table_parser import parser_functions as parser
import xlwings as xw
from tqdm import tqdm
import FinanceDataReader as fdr
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from selenium import webdriver

api_key = '08d5ae18b24d9a11b7fd67fb0d79c607f1c88464'
dart = OpenDartReader(api_key)
today = datetime.today().strftime("%Y%m%d") #오늘


dart_list = dart.list(start=today,end=today)
lists = dart_list[dart_list['report_nm'].str.contains('기업설명회')]
print(f'{today}_{len(lists)}건')

def investment_briefing(rcp_no,corp):
    
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(f'https://dart.fss.or.kr/dsaf001/main.do?rcpNo={rcp_no}')
    driver.switch_to.frame('ifrm')
    html = driver.page_source #지금 현 상태의 page source불러오기
    soup = BeautifulSoup(html,'html.parser')
    tables = soup.find_all('table')#표를 불러옴
    p=parser.make2d(tables[0])#표를 만듬
    tt=pd.DataFrame(p)
    tt = tt.T
    if '장소' in tt.iloc[0,0]:
        tt = tt[[0,1,2,3,4,5,6,7,8,9]]
        tt.columns = ['일시','장소','참가대상','목적','방법','후원기관','요약','결정일','게재일시','웹페이지']
        tt = tt.drop(index=[0,1,3])
        tt = tt.dropna(axis=1)
        tt.insert(0,'기업',corp)
        tt.insert(0,'조회날짜',today)
    
    else:
        tt['시작일']=tt[2].shift(1)
        tt['일시'] = tt['시작일']+'~'+tt[2]
        tt = tt[['일시',3,4,5,6,8,7,9,12,13]]
        tt.columns = ['일시','장소','참가대상','목적','방법','후원기관','요약','결정일','게재일시','웹페이지']
        tt = tt.iloc[[2],:]
        tt.insert(0,'기업',corp)
        tt.insert(0,'조회날짜',today)
    return tt

df = pd.DataFrame()

for i in range(len(lists)):
    try:
        # rcp_no = '20240822900430'
        # corp = 'bgf'
        rcp_no = lists['rcept_no'].values[i]
        corp = lists['flr_nm'].values[i]
        print(rcp_no,corp)
        ir_df = investment_briefing(rcp_no,corp)


        print(f'{i}_{corp} 완료')  
        df = pd.concat([df,ir_df])
    except Exception as e:
        print(f'{corp}',e)
        
file_name = f'C:\투자기록\정리\코딩\장내매수\기업설명회_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
df.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()

20241007_7건
20241007800598 한세예스24홀딩스
0_한세예스24홀딩스 완료
20241007900612 예스24
1_예스24 완료
20241007800604 한세엠케이
2_한세엠케이 완료
20241007800597 한세실업
3_한세실업 완료
20241007800499 롯데쇼핑
4_롯데쇼핑 완료
20241007900315 아셈스
5_아셈스 완료
20241007800077 POSCO홀딩스
6_POSCO홀딩스 완료


In [53]:
file_name = f'C:\투자기록\정리\코딩\장내매수\기업설명회_종합.xlsx'
writer = pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay')
df.to_excel(
    writer, 
    sheet_name='Sheet1',
    startcol = 0,
    startrow = writer.sheets['Sheet1'].max_row,
    index=False, 
    na_rep = '',      # 결측값을 ''으로 채우기
    inf_rep = '',     # 무한값을 ''으로 채우기
    header = None
    )
writer.close()
